### FFT with rotating plate

The setup was conducted as follows:


1.The laser with wavelength $\lambda$

= 660 nm was calibrated to hit the polarimeter at the end of the setup centrally.


2.A train consisting of linear polarizer LP2 (Pol.), $\frac{\lambda}{2}$
-WP 600 - 2700 nm and $\frac{\lambda}{4}$

-WP 690 - 1200 nm was assembled and placed into the laser beam.


3.By rotating the WPs of the train horizontally polarized light was set.

The angles of the optical elements in the train in the order set were $\Theta_{1}=49^{\circ}$
, $\Theta_{2}=227^{\circ}$ and $\Theta_{3}=???^{\circ}$

, because the last plate holder had no markings.

The parameters of the horizontally polarized light were: S1=1.000 ; S2=-0.002 ; S3=-0.004 ; DOP=103.9 % and power=-19.61 dBm

Annotation: The angle of LP2 (Pol.) in the train could have been set that maximum intensity is transmitted, but wasn't.


4.In order to find the angle $\alpha$
of the x-axis following procedure was used. Another linear polarizer LP3, which is here also referred to as P1, was inserted after the train and rotated until maximum intensity was measured by the polarimeter.This was the case for $\alpha_{1}=131^{\circ}$

The measured parameters were:

S1=0.999 ; S2=0.001 ; S3=0.041 ; DOP=103.7 % and power=-21.3 dBm


5.Another linear polarizer LP1, which is here also referred as P2, was inserted behind P1 just in front of the polarimeter and rotated until maximum intensity was measured by the polarimeter.This was the case for $\alpha_{2}=336^{\circ}$

The measured parameters were:

S1=1.000 ; S2=0.002 ; S3=0.002 ; DOP=103.7 % and power=-21.95 dBm

Now the angle of P2 is set to transmit horizontally polarized light most effectively and therefore this angle is assigned to the x-axis. Accordingly, in relation to the x-axis P2 is now set at $\alpha$=0.
This position is not changed during the measurement


6.All components are removed, except the train, and left-circularly polarized light intended for measurement is set.

S1=-0.003 ; S2=-0.067 ; S3=-0.997 ; DOP=98.3 % and power=-19.87 dBm


7.Behind the train we insert the $\frac{\lambda}{4}$
-WP 600 - 2700 nm in the rotating wheel and behind the wheel we insert LP1 as P2 at angle $\alpha$=0


8.The wheel is rotated from 0 to 2 𝜋
and 361 equally distanced data points are recorded

### Read in data for setup like in paper
Do experiment for left-handed polarized light

In [44]:
import numpy as np
import matplotlib.pyplot as plt

from sympy import Symbol, cos, sin, pi, sqrt, symbols
from sympy.matrices import *
from sympy.utilities.lambdify import lambdify
from sympy.interactive.printing import init_printing
from sympy.interactive import printing
printing.init_printing(use_latex=True)

from scipy.optimize import curve_fit, minimize
from scipy import fftpack
import os
from os.path import join, getsize

directories=[]

# Set some parameters for aesthetically pleasing plots 
import matplotlib as mpl
mpl.use('pdf')
 
plt.rcParams['text.latex.preamble']=[r"\usepackage{lmodern}"]
plt.rcParams['text.latex.preamble']=[r"\usepackage{siunitx}"]                               


plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=8.7)
plt.rc('ytick', labelsize=8.7)
plt.rc('axes', labelsize=8.7)

# width as measured in inkscape
width = 3.487
height = width / 1.618

<font color="red">In order to use the method with this script a measurement of circular light is necessary in order to calibrate the parameter $\beta_{0}$. After the script has run through with the measurement of circular light, which can be set below in In [45] the obtained parameter for beta_0 needs to be hardcoded in In [49] on the spot where the other hardcoded values for beta_0 are. After this step is completed any measurement can be read in In [45] and evaluated just by running the script.</font>

In [45]:
# Read in data and choose which measurement to evaluate 
for root, dirs, files in os.walk('20190521/'):
    if files != []:
        a=str(root)+str('/')+str(files[0])
        directories.append(a)

# Some names for the data files to know which polarization
# 20190515[0]horizontal polarization
# 20190515[1]vertical polarization
# 20190516[0]diagonal polarization
# 20190520[0]antidiagonal polarization
# 20190521[0]left-circular polarization
# 20190521[1]right-circular polarization
        
# Choose here file if there are multiple files within the directory
directories=directories[0:1]

data=[]
measurement_point_all=[]
angle_all=[]
s1_all=[]
s2_all=[]
s3_all=[]
dop_all=[]
power_all=[]

for i in directories:
    f=open(i)
    lines=f.readlines()
    for i in range(26):
        lines.pop(0)
    measurement_point=[]
    angle=[]
    s1=[]
    s2=[]
    s3=[]
    dop=[]
    power=[]
    for line in lines:
        measurement_point.append( line.strip().split('\t')[0] )
        angle.append( line.strip().split('\t')[1] )
        s1.append( line.strip().split('\t')[2] )
        s2.append( line.strip().split('\t')[3] )
        s3.append( line.strip().split('\t')[4] )
        dop.append( line.strip().split('\t')[5] )
        power.append( line.strip().split('\t')[6] )
    measurement_point_all.append(measurement_point)
    angle_all.append(angle)
    s1_all.append(s1)
    s2_all.append(s2)
    s3_all.append(s3)
    dop_all.append(dop)
    power_all.append(power)
    
# The value for each datapoint of the 361 angles is saved in the corresponding arrays
measurement_point=np.array(measurement_point_all).astype(np.float)  
angle=np.array(angle_all).astype(np.float)
s1=np.array(s1_all).astype(np.float) 
s2=np.array(s2_all).astype(np.float)
s3=np.array(s3_all).astype(np.float) 
dop=np.array(dop_all).astype(np.float)
power=np.array(power_all).astype(np.float)

In [46]:
#plt.figure()
#plt.plot(angle[0], power[0])
#plt.ylabel('Intensity')
#plt.xlabel('Degree °')
#plt.title(r'Measurement for $\frac{1}{2}$ WP #40')

### Calculate discrete Fourier series coefficients like in paper
For an uneven number of data points like in our case with N = 2L = 361 we can calculate the discrete Fourier series coefficients according to
$$C_{\omega, k}=\frac{2}{N} \frac{1}{1+\delta_{k0}} \sum_{i=1}^N I_{T_{i}} cos \omega_{k} \beta_{i}$$
$$S_{\omega, k}=\frac{2}{N} \frac{1}{1+\delta_{k0}} \sum_{i=1}^N I_{T_{i}} sin \omega_{k} \beta_{i}$$
with the discrete angular velocities
$$\omega_{k} = \frac{2\pi}{N} \frac{k}{\Delta \beta}$$ the angular positions $$\beta_{i}=(i-1)\Delta \beta$$ and $\Delta \beta$ being the difference in the angle for two neighboring data points.

We need to consider that due to the dependance of the discrete angular velocities on the step size, it can happen that the 2 $\omega$ and 4$\omega$ signals, which correspond to circular and linear polarizations, are not exactly included in our calculations. Therefore, we look for the discrete angular frequencies, which are closest to the requirements.

### Starting Parameters
For circular polarized light
S1=-0.003
S2=-0.067
S3=-0.997
DOP: 98.3 %
Power: -19.87 dBm

In [47]:
# Calculate discrete Fourier coefficients for an uneven number of data points N=2L+1

intensity=power[0]

L=180
N=2*L+1
index=np.linspace(0,2,361)

omega_k=[]
# Define step size
delta_beta=(index[1]-index[0])*2*np.pi
# Calculate all omega_k up to k=180, more than that is not required anyways
for i in np.arange(N):
    omega_k.append(2*np.pi*i/(N*delta_beta))
omega_k=np.array(omega_k)
# Calculate Fourier series coefficients
C_omega_k=[]
S_omega_k=[]

# Sum in definition of C_omega_k with cos terms

def sum_cos(x):
    ''' Calculate formula from above with variable x=omega_{k}'''
    sum_cos=0
    j=0
    for i in index:
        sum_cos=sum_cos+ intensity[j]*np.cos(x*i*np.pi)
        j+=1
    return sum_cos
a=sum_cos(omega_k)

# Sum in definition of S_omega_k with sin terms
def sum_sin(x):
    '''Same with sin'''
    sum_sin=0
    j=0
    for i in index:
        sum_sin+= intensity[j]*np.sin(x*i*np.pi)
        j+=1
    return sum_sin

#print(sum_sin(omega_k))
def fourier_coefficients(x):
    '''Calculate complete Fourier coefficients'''
    C_omega_k=(2/N)* sum_cos(x)
    S_omega_k=(2/N)* sum_sin(x)
    return C_omega_k, S_omega_k

# The following functions are defined in order to calculate the errors


sigma_I=0.134*10**(-6) # This is the error on the intensity previously calculated from a noise measurement

def sum_cos_inten_err(x):
    ''' Function for error on sum with cos'''
    sum_cos_error=0
    for i in index:
        sum_cos_error=sum_cos_error+ (np.cos(x*i))**2
        
    return sum_cos_error

def sum_sin_inten_err(x):
    ''' Same with sin'''
    sum_sin_error=0
    for i in index:
        sum_sin_error=sum_sin_error+ (np.sin(x*i))**2
        
    return sum_sin_error

def intensity_error(x):
    '''Total error on fourier coefficients from error on intensity'''
    C_omega_k_inten_error=np.sqrt(sum_cos_inten_err(x)*(2*sigma_I/N)**2 )
    S_omega_k_inten_error=np.sqrt(sum_sin_inten_err(x)*(2*sigma_I/N)**2 )
    return C_omega_k_inten_error, S_omega_k_inten_error


sigma_beta=0.3*np.pi/180 # Error on beta was estimated to 0.3 degree

def sum_cos_beta_err(x):
    ''' Error on cos sum for beta'''
    sum_cos=0
    j=0
    for i in index:
        sum_cos=sum_cos+ (intensity[j]*(np.sin(x*i)))**2
        j+=1
    return sum_cos

def sum_sin_beta_err(x):
    ''' Same as above with sin'''
    sum_sin=0
    j=0
    for i in index:
        sum_sin=sum_sin+ (intensity[j]*(x*np.cos(x*i)))**2
        j+=1
    return sum_sin

def beta_error(x):
    '''Total error on fourier coefficients from error on beta'''
    C_omega_k_beta_error=np.sqrt(sum_cos_beta_err(x)*(2*sigma_beta/N)**2)
    S_omega_k_beta_error=np.sqrt(sum_sin_beta_err(x)*(2*sigma_beta/N)**2)
    return C_omega_k_beta_error, S_omega_k_beta_error




# Loop to get all Fourier coefficients
Cs=[]
indices=[]
Ss=[]

Cs_inten_err=[]
Ss_inten_err=[]

Cs_beta_err=[]
Ss_beta_err=[]
# Calculate only first 11 Fourier coefficients

# Find indices, where omega_k is closest to desired integer values

for i in np.arange(11):
    indices.append(np.where(  np.absolute(omega_k-i)==np.amin(np.absolute(omega_k-i)) )[0][0] )

for i in np.arange(11):
    Cs.append(fourier_coefficients(omega_k)[0][indices[i]])
    Ss.append(fourier_coefficients(omega_k)[1][indices[i]])
    Cs_inten_err.append(intensity_error(omega_k)[0][indices[i]])
    Ss_inten_err.append(intensity_error(omega_k)[1][indices[i]])
    Cs_beta_err.append(beta_error(omega_k)[0][indices[i]])
    Ss_beta_err.append(beta_error(omega_k)[1][indices[i]])
Cs[0]=Cs[0]*0.5
#Calculating Fourier coefficients, which correspond to omega_k=2, 4
# Same method as above, did it just to check if the method is right
# Find index of entry of omega_k, which is closest to omega_k = 0,2 or 4
index_0,=np.where( omega_k==np.amin(omega_k) )
index_2,=np.where(  np.absolute(omega_k-2)==np.amin(np.absolute(omega_k-2)) )
index_4,=np.where(  np.absolute(omega_k-4)==np.amin(np.absolute(omega_k-4)) )


print('omega_k=', omega_k)
# Calculate Fourier coefficients for omega_k = 2,4
# Factor 0.5 for omega_k=0 to account for special case of fourier series coefficient formula
C0=fourier_coefficients(omega_k)[0][index_0[0]]*0.5
C2=fourier_coefficients(omega_k)[0][index_2[0]]
C4=fourier_coefficients(omega_k)[0][index_4[0]]
S2=fourier_coefficients(omega_k)[1][index_2[0]]
S4=fourier_coefficients(omega_k)[1][index_4[0]]

C0_inten_err=intensity_error(omega_k)[0][index_0[0]]*0.5
C2_inten_err=intensity_error(omega_k)[0][index_2[0]]
C4_inten_err=intensity_error(omega_k)[0][index_4[0]]
S2_inten_err=intensity_error(omega_k)[1][index_2[0]]
S4_inten_err=intensity_error(omega_k)[1][index_4[0]]

C0_beta_err=beta_error(omega_k)[0][index_0[0]]*0.5
C2_beta_err=beta_error(omega_k)[0][index_2[0]]
C4_beta_err=beta_error(omega_k)[0][index_4[0]]
S2_beta_err=beta_error(omega_k)[1][index_2[0]]
S4_beta_err=beta_error(omega_k)[1][index_4[0]]

sigma_C0=np.sqrt(C0_inten_err**2 + C0_beta_err**2  )
sigma_C2=np.sqrt(C2_inten_err**2 + C2_beta_err**2  )
sigma_C4=np.sqrt(C4_inten_err**2 + C4_beta_err**2  )
sigma_S2=np.sqrt(S2_inten_err**2 + S2_beta_err**2  )
sigma_S4=np.sqrt(S4_inten_err**2 + S4_beta_err**2  )


omega_k= [  0.           0.49861496   0.99722992   1.49584488   1.99445983
   2.49307479   2.99168975   3.49030471   3.98891967   4.48753463
   4.98614958   5.48476454   5.9833795    6.48199446   6.98060942
   7.47922438   7.97783934   8.47645429   8.97506925   9.47368421
   9.97229917  10.47091413  10.96952909  11.46814404  11.966759
  12.46537396  12.96398892  13.46260388  13.96121884  14.4598338
  14.95844875  15.45706371  15.95567867  16.45429363  16.95290859
  17.45152355  17.9501385   18.44875346  18.94736842  19.44598338
  19.94459834  20.4432133   20.94182825  21.44044321  21.93905817
  22.43767313  22.93628809  23.43490305  23.93351801  24.43213296
  24.93074792  25.42936288  25.92797784  26.4265928   26.92520776
  27.42382271  27.92243767  28.42105263  28.91966759  29.41828255
  29.91689751  30.41551247  30.91412742  31.41274238  31.91135734
  32.4099723   32.90858726  33.40720222  33.90581717  34.40443213
  34.90304709  35.40166205  35.90027701  36.39889197  36.89750693
  37

We need to regard only the 0 $\omega$, 2$\omega$ and 4$\omega$ signals, as these are the only ones contributing to the resulting intensity. As expected for horizontally polarized light the coefficients C2 and S2, which correspond to circular polarized light, are suppressed

In [48]:
print("C0=", Cs[0])
print("C2=", Cs[2]) 
print("C4=", Cs[4])
print("S2=", Ss[2])
print("S4=", Ss[4])

print("C0_inten_err=", Cs_inten_err[0])
print("C2_inten_err=", Cs_inten_err[2]) 
print("C4_inten_err=", Cs_inten_err[4])
print("S2_inten_err=", Ss_inten_err[2])
print("S4_inten_err=", Ss_inten_err[4])

print("C0_beta_err=", Cs_beta_err[0])
print("C2_beta_err=", Cs_beta_err[2]) 
print("C4_beta_err=", Cs_beta_err[4])
print("S2_beta_err=", Ss_beta_err[2])
print("S4_beta_err=", Ss_beta_err[4])


print("sigma_C0=", sigma_C0)
print("sigma_C2=", sigma_C2)
print("sigma_C4=", sigma_C4)
print("sigma_S2=", sigma_S2)
print("sigma_S4=", sigma_S4)

C0= 5.4812609692210406e-06
C2= 2.369012743492875e-06
C4= 3.078577589957762e-07
S2= -3.967523969270928e-06
S4= 6.205074329187946e-07
C0_inten_err= 1.4105263157894737e-08
C2_inten_err= 1.0580128781842025e-08
C4_inten_err= 9.897435244357923e-09
S2_inten_err= 9.328414855327814e-09
S4_inten_err= 1.0049837030380352e-08
C0_beta_err= 0.0
C2_beta_err= 2.4446808046663824e-09
C4_beta_err= 2.2330292817314315e-09
S2_beta_err= 5.114334686807493e-09
S4_beta_err= 1.0971725021631897e-08
sigma_C0= 7.052631578947369e-09
sigma_C2= 1.0858894477665132e-08
sigma_C4= 1.0146213293112283e-08
sigma_S2= 1.0638408856675087e-08
sigma_S4= 1.4878775967380771e-08


### Calculate the other Stokes parameters
\begin{align}
M=\frac{2}{1-cos\delta}\left[C_{4}cos(2\alpha + 4\beta_{0}) + S_{4}sin(2\alpha + 4\beta_{0})\right]\\
|M|=\frac{2}{1-cos\delta} \left( C_{4}^2 + S_{4}^2  \right)^{\frac{1}{2}}\\
C=\frac{2}{1-cos\delta}\left[S_{4}cos(2\alpha + 4\beta_{0}) - C_{4}sin(2\alpha + 4\beta_{0})\right]\\
S=\frac{C_{2}}{sin\delta sin(2\alpha + 4\beta_{0})}=\frac{-S_{2}}{sin \delta cos(2\alpha + 4\beta_{0})}\\
|S|= \frac{  \left( C_{2}^2 + S_{2}^2\right)^{\frac{1}{2}}}{  sin^2 \delta}\\
I=C_{0} - \frac{1+cos \delta}{1-cos \delta} \left[  C_{4} cos(4 \alpha+ 4\beta_{0}) + S_{4} sin(4\alpha +4\beta_{0})     \right]
\end{align} 
In order to obtain the values of the parameters we need to measure the angles $\alpha$ and $\beta_{0}$, where $\alpha$ can be obtained while setting the system up and $\beta_{0}$ can be obtained by measuring either strongly circularly polarized light or strongly linearly polarized light and using the relations above.

For strongly circularly polarized light we get
\begin{align}
tan(4\beta_{0} + 2\alpha)=\frac{-C_{2}}{S_{2}}
\end{align}



We also need the retardance of the half-wave-plate. Here the theoretical retardance was used. One can also take the retardance determined by one of the other methods described in the bachelor thesis

In [49]:
modulus_S=sqrt( Cs[2]**2 + Ss[2]**2  )/(np.sin(np.pi/2)**2)
modulus_M=sqrt( Cs[4]**2 + Ss[4]**2  )*4/(1-cos(np.pi/2))



# Calculate Stokes parameters from angles

# Define retardance and angles
#delta=np.pi/2
#delta from matrix fits 600-2700 QWP (theoretical)
delta=1.5233
alpha_0=0
# Formula for circularly polarized light
#beta_0=(np.arctan(-Cs[2]/Ss[2])-2*alpha)/4

# Some differentiation between cases as arctan is not unambiguous

if -Cs[2]>0:
    print('AAa')
    beta_0=(np.arctan2(-Cs[2],Ss[2])-2*alpha_0)/4
else:
    print('bbb')
    beta_0=(np.arctan2(-Cs[2],Ss[2])+2*np.pi-2*alpha_0)/4
    
#beta_0=(np.arctan(-Cs[2]/Ss[2])-2*alpha)/4
#beta_0=0.9199694623582562
#beta_0=0.5209602662591997
#beta_0=0.128
#beta_0=1.2470131178417307
#beta_0=0.07615438147045547

bbb


### Calculate Errors on parameters

In [50]:
sigma_delta=1*np.pi/180 # Errors estimated to one degree
sigma_alpha_0=1*np.pi/180

dbeta_0dC2=Ss[2]/(Cs[2]**2 + Ss[2]**2)
dbeta_0dS2=Cs[2]/(Cs[2]**2 + Ss[2]**2)
dbeta_0dalpha_0=-0.5

sigma_beta_0=((dbeta_0dC2*sigma_C2)**2 + (dbeta_0dS2*sigma_S2)**2 + (dbeta_0dalpha_0*sigma_alpha_0)**2)**(0.5)

print("sigma_beta_0=", sigma_beta_0*180/np.pi)


sigma_beta_0= 0.5176257403657308


In [51]:
# Calculate Stokes parameters from angles and Fourier coefficients
M=(2/(1-np.cos(delta)))*( Cs[4]*np.cos(2*alpha_0+4*beta_0) + Ss[4]*np.sin(2*alpha_0+4*beta_0)  )
C=(2/(1-np.cos(delta)))*( Ss[4]*np.cos(2*alpha_0+4*beta_0) - Cs[4]*np.sin(2*alpha_0+4*beta_0)  )
# Calculate S from Cs[2] and Ss[2] each to check on plausibility (see Formula)
S_first=Cs[2]/(np.sin(delta)*np.sin(2*alpha_0+4*beta_0))
S_second=-Ss[2]/(np.sin(delta)*np.cos(2*alpha_0+4*beta_0))
I=Cs[0]-((1+np.cos(delta))/(1-np.cos(delta)))*(Cs[4]*np.cos(4*alpha_0 + 4*beta_0)+Ss[4]*np.sin(4*alpha_0 + 4*beta_0))
L=np.sqrt(Cs[4]**2 +Ss[4]**2)/sin(delta/2)**2
L_other=sqrt(M**2 + C**2)
# Modulation ratio
# This formula only applicable to horizontal polarization
#eta=M*(1-np.cos(delta))/(2*I + M*np.cos(2*alpha*(1+np.cos(delta))))

In [52]:
print("C0=", Cs[0])
print("C2=", Cs[2]) 
print("C4=", Cs[4])
print("S2=", Ss[2])
print("S4=", Ss[4])
print('I=', I)
print('M/I=', (M/I) )
print('C/I=', (C/I)   )
print('S from C_{2}/I=', S_first/I)
print('S from -S_{2}/I=', S_second/I)
print('L/I',L/I)
print('L_other/I', L_other/I)

print('|S|=', modulus_S)
print('|S|/I with sign from S_{2}=', (modulus_S/I)*np.sign(S_second) )
print('|S|/I with sign from C_{2}=', (modulus_S/I)*np.sign(S_first) )

print('(L/I)**2 + (|S|/I)**2=',(L/I)**2 + (modulus_S/I)**2)

print('|M|', modulus_M)
print('|M|/I', modulus_M/I)

C0= 5.4812609692210406e-06
C2= 2.369012743492875e-06
C4= 3.078577589957762e-07
S2= -3.967523969270928e-06
S4= 6.205074329187946e-07
I= 6.121759393709424e-06
M/I= -0.19976835782017593
C/I= -0.128597582422088
S from C_{2}/I= -0.7556975220227682
S from -S_{2}/I= -0.7556975220227676
L/I 0.237581007218539
L_other/I 0.237581007218539
|S|= 4.62098126219648e-6
|S|/I with sign from S_{2}= -0.754845292832791
|S|/I with sign from C_{2}= -0.754845292832791
(L/I)**2 + (|S|/I)**2= 0.626236151102798
|M| 2.77072084218205e-6
|M|/I 0.452602048526960


For $\beta_{0}$ from the formula for circular polarized light we get

In [53]:
display('beta_{0}', beta_0)
print(beta_0*180/np.pi)
print('-Cs2/Ss2',-Cs[2]/Ss[2])

print( (np.arctan2(1,0)*180/np.pi) + 2*180)

'beta_{0}'

52.71036747404754
-Cs2/Ss2 0.5971010539170617
450.0


### Calculate Errors on Stokes parameters

In [54]:
print("C0=", Cs[0])
print("C2=", Cs[2]) 
print("C4=", Cs[4])
print("S2=", Ss[2])
print("S4=", Ss[4])
print('I=', I)
print('M/I=', (M/I) )
print('C/I=', (C/I)   )
print('S from C_{2}/I=', S_first/I)
print('S from -S_{2}/I=', S_second/I)
print('L/I',L/I)
print('L_other/I', L_other/I)

print('|S|=', modulus_S)
print('|S|/I with sign from S_{2}=', (modulus_S/I)*np.sign(S_second) )
print('|S|/I with sign from C_{2}=', (modulus_S/I)*np.sign(S_first) )

print('(L/I)**2 + (|S|/I)**2=',(L/I)**2 + (modulus_S/I)**2)

print('|M|', modulus_M)
print('|M|/I', modulus_M/I)

# Error on intensity

alpha_tilde=0
sigma_alpha_tilde=0


dIdC0=1
dIddelta=(2*np.sin(delta))*(C4*np.cos(4*alpha_0 + 4*alpha_tilde - 4*beta_0) + S4*np.sin(4*alpha_0 + 4*alpha_tilde - 4*beta_0))/((1-np.cos(delta))**2)
dIdC4=(np.cos(delta)+1)*(np.cos(4*alpha_0 + 4*alpha_tilde - 4*beta_0))/(np.cos(delta)-1)
dIdS4=(np.cos(delta)+1)*(np.sin(4*alpha_0 + 4*alpha_tilde - 4*beta_0))/(np.cos(delta)-1)
dIdalpha_0=4*(np.cos(delta)+1)*((S4*np.cos(4*alpha_0 + 4*alpha_tilde - 4*beta_0) - C4*np.sin(4*alpha_0 + 4*alpha_tilde - 4*beta_0)))/(np.cos(delta)-1)
dIdalpha_tilde=4*(np.cos(delta)+1)*((S4*np.cos(4*alpha_0 + 4*alpha_tilde - 4*beta_0) - C4*np.sin(4*alpha_0 + 4*alpha_tilde - 4*beta_0)))/(np.cos(delta)-1)
dIdbeta_0=-4*(np.cos(delta)+1)*((S4*np.cos(4*alpha_0 + 4*alpha_tilde - 4*beta_0) - C4*np.sin(4*alpha_0 + 4*alpha_tilde - 4*beta_0)))/(np.cos(delta)-1)

sigma_I= ((dIdC0*sigma_C0)**2 + (dIddelta*sigma_delta)**2 + (dIdC4*sigma_C4)**2 + (dIdS4*sigma_S4)**2 + (dIdalpha_0*sigma_alpha_0)**2 + (dIdalpha_tilde*sigma_alpha_tilde)**2+ (dIdbeta_0*sigma_beta_0)**2)**(1/2)

print("sigma_I",sigma_I)

# Error on M

dMddelta=-(2*np.sin(delta))*(C4*np.cos(2*alpha_0 +2*alpha_tilde-4*beta_0) + S4*np.sin(2*alpha_0 +2*alpha_tilde -4*beta_0))/(1-np.cos(delta))**2
dMdC4=-2*np.cos(2*alpha_0 +2*alpha_tilde-4*beta_0)/(np.cos(delta)-1)
dMdS4=-2*np.sin(2*alpha_0 +2*alpha_tilde-4*beta_0)/(np.cos(delta)-1)
dMdalpha_0=4*((C4*np.sin(2*alpha_0 +2*alpha_tilde-4*beta_0) - S4*np.cos(2*alpha_0 +2*alpha_tilde -4*beta_0)))/(np.cos(delta)-1)
dMdalpha_tilde=4*((C4*np.sin(2*alpha_0 +2*alpha_tilde-4*beta_0) - S4*np.cos(2*alpha_0 +2*alpha_tilde -4*beta_0)))/(np.cos(delta)-1)
dMdbeta_0=8*((S4*np.cos(2*alpha_0 +2*alpha_tilde-4*beta_0) - C4*np.sin(2*alpha_0 +2*alpha_tilde -4*beta_0)))/(np.cos(delta)-1)

sigma_M=( (dMddelta*sigma_delta)**2 + (dMdC4*sigma_C4)**2 +(dMdS4*sigma_S4)**2 + (dMdalpha_0*sigma_alpha_0)**2 + (dMdalpha_tilde*sigma_alpha_tilde)**2 + (dMdbeta_0*sigma_beta_0)**2)**(0.5)

print("sigma_M",sigma_M)
print(dMddelta*sigma_delta)
print(dMdC4*sigma_C4)
print(dMdS4*sigma_S4)
print(dMdalpha_0*sigma_alpha_0)
print(dMdalpha_tilde*sigma_alpha_tilde)
print(dMdbeta_0*sigma_beta_0)
sigma_M_over_I=((sigma_M/I)**2 + (-M*sigma_I/I**2)**2)**(0.5)

print("sigma_M_over_I",sigma_M_over_I)

# Error on C

dCddelta=-(2*np.sin(delta))*(S4*np.cos(2*alpha_0 +2*alpha_tilde-4*beta_0) - C4*np.sin(2*alpha_0 +2*alpha_tilde -4*beta_0))/(1-np.cos(delta))**2
dCdC4=2*np.sin(2*alpha_0 +2*alpha_tilde-4*beta_0)/(np.cos(delta)-1)
dCdS4=-2*np.cos(2*alpha_0 +2*alpha_tilde-4*beta_0)/(np.cos(delta)-1)
dCdalpha_0=4*((C4*np.cos(2*alpha_0 +2*alpha_tilde-4*beta_0) +S4*np.sin(2*alpha_0 +2*alpha_tilde -4*beta_0)))/(np.cos(delta)-1)
dCdalpha_tilde=4*((C4*np.cos(2*alpha_0 +2*alpha_tilde-4*beta_0) +S4*np.sin(2*alpha_0 +2*alpha_tilde -4*beta_0)))/(np.cos(delta)-1)
dCdbeta_0=-8*((C4*np.cos(2*alpha_0 +2*alpha_tilde-4*beta_0) + S4*np.sin(2*alpha_0 +2*alpha_tilde -4*beta_0)))/(np.cos(delta)-1)

sigma_C=( (dCddelta*sigma_delta)**2 + (dCdC4*sigma_C4)**2 +(dCdS4*sigma_S4)**2 + (dCdalpha_0*sigma_alpha_0)**2 + (dCdalpha_tilde*sigma_alpha_tilde)**2 + (dCdbeta_0*sigma_beta_0)**2)**(0.5)

print("sigma_C",sigma_C)
print(dCddelta*sigma_delta)
print(dCdC4*sigma_C4)
print(dCdS4*sigma_S4)
print(dCdalpha_0*sigma_alpha_0)
print(dCdalpha_tilde*sigma_alpha_tilde)
print(dCdbeta_0*sigma_beta_0)

sigma_C_over_I=((sigma_C/I)**2 + (-C*sigma_I/I**2)**2)**(0.5)

print("sigma_C_over_I",sigma_C_over_I)

# Error on S from C2

dS_C2dC2=1/((np.sin(delta))*(np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)))
dS_C2ddelta=-C2*np.cos(delta)/(np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)*sin(delta)**2)
dS_C2dalpha_0=-2*C2*np.cos(2*alpha_0 + 2*alpha_tilde - 2*beta_0)/(np.sin(delta)*np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)**2)
dS_C2dalpha_tilde=-2*C2*np.cos(2*alpha_0 + 2*alpha_tilde - 2*beta_0)/(np.sin(delta)*np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)**2)
dS_C2dbeta_0=2*C2*np.cos(2*alpha_0 + 2*alpha_tilde - 2*beta_0)/(np.sin(delta)*np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)**2)

sigma_S_C2=( (dS_C2ddelta*sigma_delta)**2 + (dS_C2dC2*sigma_C2)**2  + (dS_C2dalpha_0*sigma_alpha_0)**2 + (dS_C2dalpha_tilde*sigma_alpha_tilde)**2 + (dS_C2dbeta_0*sigma_beta_0)**2)**(0.5)

print("sigma_S_C2",sigma_S_C2)
print(dS_C2ddelta*sigma_delta)
print(dS_C2dC2*sigma_C2)
print(dS_C2dalpha_0*sigma_alpha_0)
print(dS_C2dalpha_tilde*sigma_alpha_tilde)
print(dS_C2dbeta_0*sigma_beta_0)
sigma_S_C2_over_I=((sigma_S_C2/I)**2 + (-S_first*sigma_I/I**2)**2)**(0.5)

print("sigma_S_C2_over_I",sigma_S_C2_over_I)


# Error on S from S2

dS_S2dS2=-1/((np.sin(delta))*(np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)))
dS_S2ddelta=S2*np.cos(delta)/(np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)*sin(delta)**2)
dS_S2dalpha_0=2*S2*np.cos(2*alpha_0 + 2*alpha_tilde - 2*beta_0)/(np.sin(delta)*np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)**2)
dS_S2dalpha_tilde=2*S2*np.cos(2*alpha_0 + 2*alpha_tilde - 2*beta_0)/(np.sin(delta)*np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)**2)
dS_S2dbeta_0=-2*S2*np.cos(2*alpha_0 + 2*alpha_tilde - 2*beta_0)/(np.sin(delta)*np.sin(2*alpha_0 + 2*alpha_tilde - 2*beta_0)**2)

sigma_S_S2=( (dS_S2ddelta*sigma_delta)**2 + (dS_S2dS2*sigma_S2)**2  + (dS_S2dalpha_0*sigma_alpha_0)**2 + (dS_S2dalpha_tilde*sigma_alpha_tilde)**2 + (dS_S2dbeta_0*sigma_beta_0)**2)**(0.5)

print("sigma_S_S2",sigma_S_S2)

sigma_S_S2_over_I=((sigma_S_S2/I)**2 + (-S_second*sigma_I/I**2)**2)**(0.5)

print("sigma_S_S2_over_I",sigma_S_S2_over_I)

# Error on |M|

dmodMdC4=-2*C4/(((C4**2 + S4**2)**(0.5))*(np.cos(delta)-1))
dmodMdS4=-2*S4/(((C4**2 + S4**2)**(0.5))*(np.cos(delta)-1))
dmodMddelta=(-2*np.sin(delta)*(C4**2 + S4**2)**(0.5))/(1-np.cos(delta))**2

sigma_modM=((dmodMdC4*sigma_C4)**2 + (dmodMdS4*sigma_S4)**2 + (dmodMddelta*sigma_delta)**2)**(0.5)

print("sigma_modM",sigma_modM)

sigma_modM_over_I=((sigma_modM/I)**2 + (-modulus_M*sigma_I/I**2)**2)**(0.5)

print("sigma_modM_over_I",sigma_modM_over_I)

# Error on |S|

dmodSdC2=C2/((np.sin(delta)**2)*(C2**2 + S2**2)**(0.5))
dmodSdS2=S2/((np.sin(delta)**2)*(C2**2 + S2**2)**(0.5))
dmodSddelta=(-2*np.cos(delta)*(C2**2 + S2**2)**(0.5))/(np.sin(delta)**3)

sigma_modS=((dmodSdC2*sigma_C2)**2 + (dmodSdS2*sigma_S2)**2 + (dmodSddelta*sigma_delta)**2)**(0.5)


print("sigma_modS=",sigma_modS)
print(dmodSdC2*sigma_C2)
print(dmodSdS2*sigma_S2)
print(dmodSddelta*sigma_delta)
sigma_modS_over_I=((sigma_modS/I)**2 + (-modulus_S*sigma_I/I**2)**2)**(0.5)

print("sigma_modS_over_I",sigma_modS_over_I)
print(sigma_modS/I)
print((-modulus_S*sigma_I/I**2)**2)
# Error on |L|

dLdC4=C4/((np.sin(delta/2)**2)*(C4**2 + S4**2)**(0.5))
dLdS4=S4/((np.sin(delta/2)**2)*(C4**2 + S4**2)**(0.5))
dLddelta=(-(C4**2 + S4**2)**(0.5))*np.cos(delta/2)/(np.sin(delta/2)**3)

sigma_L=np.sqrt((dLdC4*sigma_C4)**2 + (dLdS4*sigma_S4)**2 + (dLddelta*sigma_delta)**2)

print('sigma_L=', sigma_L)
print(dLdC4*sigma_C4)
print(dLdS4*sigma_S4)
print(dLddelta*sigma_delta)

sigma_L_over_I=((sigma_L/I)**2 + (-L*sigma_I/(I)**2)**2)**(0.5)

print("sigma_L_over_I=", sigma_L_over_I)
print(sigma_L/I)
print(-L*sigma_I/(I)**2)

C0= 5.4812609692210406e-06
C2= 2.369012743492875e-06
C4= 3.078577589957762e-07
S2= -3.967523969270928e-06
S4= 6.205074329187946e-07
I= 6.121759393709424e-06
M/I= -0.19976835782017593
C/I= -0.128597582422088
S from C_{2}/I= -0.7556975220227682
S from -S_{2}/I= -0.7556975220227676
L/I 0.237581007218539
L_other/I 0.237581007218539
|S|= 4.62098126219648e-6
|S|/I with sign from S_{2}= -0.754845292832791
|S|/I with sign from C_{2}= -0.754845292832791
(L/I)**2 + (|S|/I)**2= 0.626236151102798
|M| 2.77072084218205e-6
|M|/I 0.452602048526960
sigma_I 6.14838458842786e-08
sigma_M 7.683088806468898e-08
-2.06709039072503e-09
-1.829129839644108e-08
1.6016056389441646e-08
-5.061531863768969e-08
-0.0
5.2399583567363006e-08
sigma_M_over_I 0.012709820644806389
sigma_C 3.9689865718275105e-08
2.6539157516910277e-08
-1.092175355002642e-08
-2.6823024820295015e-08
-3.942342130219762e-09
-0.0
4.0813155278600336e-09
sigma_C_over_I 0.0066108042988762645
sigma_S_C2 2.90316555249090e-8
2.04100886937685e-9
-1.12771

### Comparison computed Stokes parameters and measured Stokes parameters

$$\vec{S}_{Meas.}=\left(\begin{array}{c} 1 \\ -0.003 \\ -0.067 \\ -0.997 \end{array}\right)$$

In [55]:
print("C0=", Cs[0])
print("C2=", Cs[2]) 
print("C4=", Cs[4])
print("S2=", Ss[2])
print("S4=", Ss[4])
print('I=', I)
print('M/I=', (M/I) )
print('C/I=', (C/I)   )
print('S from C_{2}/I=', S_first/I)
print('S from -S_{2}/I=', S_second/I)
print('L/I',L/I)
print('L_other/I', L_other/I)

print('|S|=', modulus_S)
print('|S|/I with sign from S_{2}=', (modulus_S/I)*np.sign(S_second) )
print('|S|/I with sign from C_{2}=', (modulus_S/I)*np.sign(S_first) )

print('(L/I)**2 + (|S|/I)**2=',(L/I)**2 + (modulus_S/I)**2)

print('|M|', modulus_M)
print('|M|/I', modulus_M/I)

C0= 5.4812609692210406e-06
C2= 2.369012743492875e-06
C4= 3.078577589957762e-07
S2= -3.967523969270928e-06
S4= 6.205074329187946e-07
I= 6.121759393709424e-06
M/I= -0.19976835782017593
C/I= -0.128597582422088
S from C_{2}/I= -0.7556975220227682
S from -S_{2}/I= -0.7556975220227676
L/I 0.237581007218539
L_other/I 0.237581007218539
|S|= 4.62098126219648e-6
|S|/I with sign from S_{2}= -0.754845292832791
|S|/I with sign from C_{2}= -0.754845292832791
(L/I)**2 + (|S|/I)**2= 0.626236151102798
|M| 2.77072084218205e-6
|M|/I 0.452602048526960


We see that the relative Stokes parameters correspond to the expectations. The sign of M is wrong, but due to the small value it could be just a fluctuation around zero.

The magnitude of S can always be computed without knowing the angles $\alpha$ and $\beta_{0}$

In [56]:
print('|S|=', modulus_S)
print('Normalized |S| with sign from S_{2}=', (modulus_S/I)*np.sign(S_second) )
print('Normalized |S| with sign from C_{2}=', (modulus_S/I)*np.sign(S_first) )

|S|= 4.62098126219648e-6
Normalized |S| with sign from S_{2}= -0.754845292832791
Normalized |S| with sign from C_{2}= -0.754845292832791


From that we have also a good correspondence.

Absolute values for completeness.

In [57]:
print('M=', M)
print('C=', C)
print('S_first=', S_first)
print('S_second=', S_second)
print('I=', I)

M= -1.2229338210515675e-06
C= -7.87243458200739e-07
S_first= -4.626198404245815e-06
S_second= -4.626198404245812e-06
I= 6.121759393709424e-06


In [58]:
z=0


fig, ax=plt.subplots()
plt.plot(angle[z], power[z]*10**6, label='Measurement')

def I_T(x):
    return Cs[0]+Cs[2]*np.cos(2*x*np.pi/180)+Cs[4]*np.cos(4*x*np.pi/180)+Ss[2]*np.sin(2*x*np.pi/180)+Ss[4]*np.sin(4*x*np.pi/180)
    
plt.plot(angle[z], I_T(np.array(angle[z]))*10**(6), '--', label='From discrete FFT' )
ax.set_xlabel(r'$\Theta$ (deg)')
ax.set_ylabel(r'Intensity ($\mu W$)')
ax.legend()
#plt.title('Intensity_0508_175328_E-light_beta_0_4')
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)
#fig.savefig('Intensity_0515_173910_H-light_beta_0_52.pdf')

### Modulation ratio

As an additional test we could look at the modulation ratio, which depends on some of the calculated parameters, to check on the plausibility of the result. This is only applicable, if we have linear polarization. We can only look at $\eta$ from the intensity 

\begin{align} \eta = \frac{I_{max} - I_{min}}{I_{max} + I_{min}} = \frac{M(1-cos\delta)}{2I + Mcos2\alpha(1+ cos\delta)} \end{align}



In [59]:
eta_intensity=(np.amax(intensity)-np.amin(intensity))/(np.amax(intensity)+np.amin(intensity))

print('eta from intensity', eta_intensity)
#print('eta from parameters(not applicable here)', eta)

eta from intensity 0.9711853877137241


### Plotting the coefficients

We expect strong 0 and 2 $\omega $ signals for circularly polarized light, which is fulfilled in this case.

In [60]:
# Produce plots like in paper
#%%
t = np.arange(11)


fig, axs = plt.subplots(1, 2, sharey=True)
# Remove horizontal space between axes

fig.subplots_adjust(wspace=0.2)

# Plot each graph, and manually set the y tick values
axs[0].stem(t, Ss)
#axs[0].set_yscale('log')
axs[0].text(0.8,-0.1, 'Angular Frequency', fontsize=12, transform=axs[0].transAxes)
axs[0].text(0.6,0.7, '(Sine coeff.)', fontsize=10, transform=axs[0].transAxes)

axs[0].set_ylabel('Amplitude', fontsize=12)
axs[0].set_xticks(ticks=np.arange(0,11,2))
axs[0].set_xticks(ticks=np.arange(0,11,2))
axs[0].spines['right'].set_visible(False)

axs[1].stem(t, Cs)
axs[1].text(0.4,0.7, '(Cosine coeff.)', fontsize=10, transform=axs[1].transAxes)
#axs[1].set_yticks([])
axs[1].set_xticks(ticks=np.arange(0,11,2))
axs[1].spines['left'].set_visible(False)
plt.suptitle('Fourier series coefficients')

Text(0.5, 0.98, 'Fourier series coefficients')

### Calculate Stokes Parameter using FFT on intensity

Do a FFT on the intensity to see if the sought-after frequencies are present

In [61]:



# Number of sample points N from above
# Sample spacing i times omega

A=fftpack.rfft(intensity)




plt.figure(figsize = (12,6))
plt.title('FFT from intensity')
# Plot first 10 omegas
plt.plot(omega_k[0:10], np.abs(A)[0:10], 'o')
plt.xlabel(r'Angular Frequency [$\omega$]')
plt.ylabel('Amplitude')
plt.axvline(x=2  )
plt.axvline(x= 4  )

# Use the same formulas again